# Import dependencies

In [1]:
import numpy as np
import math 
import re
import pandas as pd
from bs4 import BeautifulSoup
# from google.colab import drive

In [2]:
from tensorflow.keras import layers
import tensorflow_datasets as tfds

# Data Preprocessing

In [3]:
#Loading Data
#for train data
cols = ["sentiment","id","date","query","user","text"]
train_data = pd.read_csv("train.csv",header= None,names= cols,
                        engine ="python",encoding ="latin")

#for test data
test_data = pd.read_csv("test.csv",header= None,names= cols,
                        engine ="python",encoding ="latin")

In [4]:
train_data.head(3)
data = train_data

# cleaning

In [5]:
data.drop(["id","date","query","user"],axis=1,inplace = True) 

In [6]:
def clean_tweet(tweet):
    tweet = BeautifulSoup(tweet,'lxml').get_text()
    #remove the @ ,letter and number 
    tweet = re.sub(r"@[A-Za-z0-9]", ' ',tweet) 
    #remove the urls
    tweet = re.sub(r"https?://[A-Za-z0-9./]+",' ',tweet)
    #
    tweet = re.sub(r"[^a-zA-Z.!']",' ',tweet)
    #
    tweet = re.sub(r" +",' ',tweet)
    #
    return tweet

In [7]:
data_clean = [clean_tweet(tweet) for tweet in data.text]

C:\Users\gaura\anaconda3\lib\site-packages\bs4\__init__.py:314: UserWarning: "b' i just received my G8 viola exam.. and its... well... .. disappointing.. :\\..'" looks like a filename, not markup. You should probably open this file and pass the filehandle into Beautiful Soup.
  ' Beautiful Soup.' % markup)
C:\Users\gaura\anaconda3\lib\site-packages\bs4\__init__.py:314: UserWarning: "b'E3 ON PLAYSTATION HOME IN ABOUT AN HOUR!!!!!!!!!! \\../  \\../'" looks like a filename, not markup. You should probably open this file and pass the filehandle into Beautiful Soup.
  ' Beautiful Soup.' % markup)


In [8]:
data_labels = data.sentiment.values
data_labels[data_labels == 4] = 1
set(data_labels)

{0, 1}

# Tokenization

In [9]:
tokenizer = tfds.features.text.SubwordTextEncoder.build_from_corpus(
    data_clean, target_vocab_size=2**16
)

data_inputs = [tokenizer.encode(sentence) for sentence in data_clean]


# padding 

In [11]:
import tensorflow as tf
MAX_LEN = max([len(sentence) for sentence in data_inputs])
data_inputs = tf.keras.preprocessing.sequence.pad_sequences(data_inputs,
                                                            value=0,
                                                            padding="post",
                                                            maxlen=MAX_LEN)

# splitting trainning and test set

In [12]:
test_idx = np.random.randint(0, 800000, 8000)
test_idx = np.concatenate((test_idx, test_idx+800000))

In [13]:
test_inputs = data_inputs[test_idx]
test_labels = data_labels[test_idx]
train_inputs = np.delete(data_inputs, test_idx, axis=0)
train_labels = np.delete(data_labels, test_idx)

# Modelling  

In [14]:
class DCNN(tf.keras.Model):
    
    def __init__(self,
                 vocab_size,
                 emb_dim=128,
                 nb_filters=50,
                 FFN_units=512,
                 nb_classes=2,
                 dropout_rate=0.1,
                 training=False,
                 name="dcnn"):
        super(DCNN, self).__init__(name=name)
        
        self.embedding = layers.Embedding(vocab_size,
                                          emb_dim)
        self.bigram = layers.Conv1D(filters=nb_filters,
                                    kernel_size=2,
                                    padding="valid",
                                    activation="relu")
        self.trigram = layers.Conv1D(filters=nb_filters,
                                     kernel_size=3,
                                     padding="valid",
                                     activation="relu")
        self.fourgram = layers.Conv1D(filters=nb_filters,
                                      kernel_size=4,
                                      padding="valid",
                                      activation="relu")
        self.pool = layers.GlobalMaxPool1D() # no training variable so we can
                                             # use the same layer for each
                                             # pooling step
        self.dense_1 = layers.Dense(units=FFN_units, activation="relu")
        self.dropout = layers.Dropout(rate=dropout_rate)
        if nb_classes == 2:
            self.last_dense = layers.Dense(units=1,
                                           activation="sigmoid")
        else:
            self.last_dense = layers.Dense(units=nb_classes,
                                           activation="softmax")
    
    def call(self, inputs, training):
        x = self.embedding(inputs)
        x_1 = self.bigram(x)
        x_1 = self.pool(x_1)
        x_2 = self.trigram(x)
        x_2 = self.pool(x_2)
        x_3 = self.fourgram(x)
        x_3 = self.pool(x_3)
        
        merged = tf.concat([x_1, x_2, x_3], axis=-1) # (batch_size, 3 * nb_filters)
        merged = self.dense_1(merged)
        merged = self.dropout(merged, training)
        output = self.last_dense(merged)
        
        return output

# APPLICATION 

## configuartion 

In [15]:
VOCAB_SIZE = tokenizer.vocab_size

EMB_DIM = 200
NB_FILTERS = 100
FFN_UNITS = 256
NB_CLASSES = 2#len(set(train_labels))

DROPOUT_RATE = 0.2

BATCH_SIZE = 32
NB_EPOCHS = 5

## Trainning

In [16]:
Dcnn = DCNN(vocab_size=VOCAB_SIZE,
            emb_dim=EMB_DIM,
            nb_filters=NB_FILTERS,
            FFN_units=FFN_UNITS,
            nb_classes=NB_CLASSES,
            dropout_rate=DROPOUT_RATE)

In [17]:
if NB_CLASSES == 2:
    Dcnn.compile(loss="binary_crossentropy",
                 optimizer="adam",
                 metrics=["accuracy"])
else:
    Dcnn.compile(loss="sparse_categorical_crossentropy",
                 optimizer="adam",
                 metrics=["sparse_categorical_accuracy"])

In [18]:
checkpoint_path = "C:\\Users\\gaura\\Desktop\\Modern Natural Language Processing in Python\\sentimental_Analysis\\ckpt"

ckpt = tf.train.Checkpoint(Dcnn=Dcnn)

ckpt_manager = tf.train.CheckpointManager(ckpt, checkpoint_path, max_to_keep=5)

if ckpt_manager.latest_checkpoint:
    ckpt.restore(ckpt_manager.latest_checkpoint)
    print("Latest checkpoint restored!!")

In [19]:
Dcnn.fit(train_inputs,
         train_labels,
         batch_size=BATCH_SIZE,
         epochs=NB_EPOCHS)
ckpt_manager.save()

Epoch 1/5
49503/49503 [==============================] - 8892s 180ms/step - loss: 0.4012 - accuracy: 0.8183
Epoch 2/5
49503/49503 [==============================] - 12124s 245ms/step - loss: 0.3323 - accuracy: 0.8575
Epoch 3/5
49503/49503 [==============================] - 14841s 300ms/step - loss: 0.2715 - accuracy: 0.8886
Epoch 4/5
49503/49503 [==============================] - 8939s 181ms/step - loss: 0.2083 - accuracy: 0.9181
Epoch 5/5
49503/49503 [==============================] - 11868s 240ms/step - loss: 0.1554 - accuracy: 0.9407


'C:\\Users\\gaura\\Desktop\\Modern Natural Language Processing in Python\\sentimental_Analysis\\ckpt\\ckpt-1'

## Evaluation

In [20]:
results = Dcnn.evaluate(test_inputs, test_labels, batch_size=BATCH_SIZE)
print(results)

500/500 [==============================] - 10s 20ms/step - loss: 0.4837 - accuracy: 0.81500s - loss: 0
[0.4837248921394348, 0.8149999976158142]


In [22]:
Dcnn(np.array([tokenizer.encode("bad teacher")]), training=False).numpy()

InvalidArgumentError: Computed output size would be negative: -1 [input_size: 2, effective_filter_size: 4, stride: 1] [Op:Conv2D]

In [23]:
tokenizer.encode("bad")

[652]